<a href="https://colab.research.google.com/github/priyanshsoni19/MajorProject/blob/main/Major_Project_ANN_PSO_DoS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Installing Libraries

In [ ]:
!pip install pyswarms
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.model_selection
from sklearn.datasets import load_iris
# Import PySwarms
import pyswarms as ps
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

     |████████████████████████████████| 102kB 5.4MB/s 



## Processing Dataset



In [ ]:
!wget "http://205.174.165.80/CICDataset/NSL-KDD/Dataset/NSL-KDD.zip"

--2020-12-02 06:05:19--  http://205.174.165.80/CICDataset/NSL-KDD/Dataset/NSL-KDD.zip
Connecting to 205.174.165.80:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6598776 (6.3M) [application/zip]
Saving to: ‘NSL-KDD.zip’

NSL-KDD.zip         100%[===================>]   6.29M  2.43MB/s    in 2.6s    

2020-12-02 06:05:25 (2.43 MB/s) - ‘NSL-KDD.zip’ saved [6598776/6598776]



In [ ]:
!unzip "NSL-KDD.zip"

Archive:  NSL-KDD.zip
  inflating: index.html              
  inflating: KDDTest1.jpg            
  inflating: KDDTest-21.arff         
  inflating: KDDTest-21.txt          
  inflating: KDDTest+.arff           
  inflating: KDDTest+.txt            
  inflating: KDDTrain1.jpg           
  inflating: KDDTrain+.arff          
  inflating: KDDTrain+.txt           
  inflating: KDDTrain+_20Percent.arff  
  inflating: KDDTrain+_20Percent.txt  


In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("KDDTrain+.txt", names=['duration',
'protocol_type',
'service',
'flag',
'src_bytes',
'dst_bytes',
'land',
'wrong_fragment',
'urgent',
'hot',
'num_failed_logins',
'logged_in',
'num_compromised',
'root_shell',
'su_attempted',
'num_root',
'num_file_creations',
'num_shells',
'num_access_files',
'num_outbound_cmds',
'is_host_login',
'is_guest_login',
'count',
'srv_count',
'serror_rate',
'srv_serror_rate',
'rerror_rate',
'srv_rerror_rate',
'same_srv_rate',
'diff_srv_rate',
'srv_diff_host_rate',
'dst_host_count',
'dst_host_srv_count',
'dst_host_same_srv_rate',
'dst_host_diff_srv_rate',
'dst_host_same_src_port_rate',
'dst_host_srv_diff_host_rate',
'dst_host_serror_rate',
'dst_host_srv_serror_rate',
'dst_host_rerror_rate',
'dst_host_srv_rerror_rate',
'label'], index_col=False)

##One-Hot Encoding On Dataset

In [ ]:

# Performing One-Hot Encoding on only three columns, 'protocol type', 'service', 'flag'
one_hot_df = pd.get_dummies(df, columns=['protocol_type','service', 'flag'])
one_hot_df


,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label,protocol_type_icmp,...,service_nnsp,service_nntp,service_ntp_u,service_other,service_pm_dump,service_pop_2,service_pop_3,service_printer,service_private,service_red_i,service_remote_job,service_rje,service_shell,service_smtp,service_sql_net,service_ssh,service_sunrpc,service_supdup,service_systat,service_telnet,service_tftp_u,service_tim_i,service_time,service_urh_i,service_urp_i,service_uucp,service_uucp_path,service_vmnet,service_whois,flag_OTH,flag_REJ,flag_RSTO,flag_RSTOS0,flag_RSTR,flag_S0,flag_S1,flag_S2,flag_S3,flag_SF,flag_SH
0,0,491,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.00,0.00,0.00,150,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,0,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13,1,0.0,0.0,0.0,0.0,0.08,0.15,0.00,255,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,123,6,1.0,1.0,0.0,0.0,0.05,0.07,0.00,255,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune,0,...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,0,232,8153,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,5,0.2,0.2,0.0,0.0,1.00,0.00,0.00,30,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,0,199,420,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,30,32,0.0,0.0,0.0,0.0,1.00,0.00,0.09,255,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125968,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,184,25,1.0,1.0,0.0,0.0,0.14,0.06,0.00,255,25,0.10,0.06,0.00,0.00,1.00,1.00,0.00,0.00,neptune,0,...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
125969,8,105,145,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.00,0.00,0.00,255,244,0.96,0.01,0.01,0.00,0.00,0.00,0.00,0.00,normal,0,...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
125970,0,2231,384,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.00,0.00,0.00,255,30,0.12,0.06,0.00,0.00,0.72,0.00,0.01,0.00,normal,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
125971,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,144,8,1.0,1.0,0.0,0.0,0.06,0.05,0.00,255,8,0.03,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


Types of Attacks

- DoS - neptune, teardrop, nmap, smurf, pod, back, land

- R2L - warezclient, guess_passwd, ftp_write, phf, warezmaster, multihop, imap, spy

- Probe - ipsweep, portsweep, satan

- U2R - perl, rootkit, buffer_overflow, loadmodule

##Dataset Splitting into Subsets

In [ ]:
Normaldf=one_hot_df[(one_hot_df.label=='normal')]
DoSdf=one_hot_df[(one_hot_df.label=='neptune') | (one_hot_df.label=='teardrop') | (one_hot_df.label=='nmap') | (one_hot_df.label=='smurf') |  (one_hot_df.label=='pod') | (one_hot_df.label=='back') | (one_hot_df.label=='land')]
R2Ldf=one_hot_df[(one_hot_df.label=='warezclient') | (one_hot_df.label=='guess_passwd') | (one_hot_df.label=='ftp_write') | (one_hot_df.label=='phf') |  (one_hot_df.label=='warezmaster') | (one_hot_df.label=='multihop') | (one_hot_df.label=='imap') | (one_hot_df.label=='spy')]
Probedf=one_hot_df[(one_hot_df.label=='ipsweep') | (one_hot_df.label=='portsweep') | (one_hot_df.label=='satan')]
U2Rdf=one_hot_df[(one_hot_df.label=='perl') | (one_hot_df.label=='rootkit') | (one_hot_df.label=='buffer_overflow') | (one_hot_df.label=='loadmodule')]

print(Normaldf.shape)
print(DoSdf.shape)
print(R2Ldf.shape)
print (Probedf.shape)
print (U2Rdf.shape)

(67343, 123)
(47420, 123)
(995, 123)
(10163, 123)
(52, 123)


## Making Training Sets

In [ ]:
X=pd.concat([Normaldf.sample(2000), DoSdf.sample(100)], axis=0) #2100
Train_R2L=pd.concat([Normaldf.sample(2000), R2Ldf.sample(50)], axis=0) #2050
Train_Probe=pd.concat([Normaldf.sample(2000), Probedf.sample(100)], axis=0) #2100
Train_U2R=pd.concat([Normaldf.sample(2000), U2Rdf.sample(50)], axis=0) #2050
Train_Mixed=pd.concat([Normaldf.sample(9000), DoSdf.sample(100), R2Ldf.sample(50), Probedf.sample(100), U2Rdf.sample(50) ], axis=0) #total 9300


In [ ]:
#X, X_test, y, y_test = sklearn.model_selection.train_test_split(X, y, test_size = 0.3, random_state = 2)

In [ ]:
#Dropping Class Labels
Y = X['label']
#CL_Train_R2L = Train_R2L['label']
#CL_Train_Probe = Train_Probe['label']
#CL_Train_U2R = Train_U2R['label']
#CL_Train_Mixed = Train_Mixed['label']

X = X.drop(['label'], axis=1)
#Train_R2L = Train_R2L.drop(['label'], axis=1)
#Train_Probe = Train_Probe.drop(['label'], axis=1)
#Train_U2R = Train_U2R.drop(['label'], axis=1)
#Train_Mixed = Train_Mixed.drop(['label'], axis=1)

In [ ]:
Y

100541     normal
45494      normal
56036      normal
25262      normal
10405      normal
           ...   
98257     neptune
111179    neptune
13801     neptune
92521       smurf
61108     neptune
Name: label, Length: 2100, dtype: object

In [ ]:
X = X.iloc[:, :].values
print(X.shape)

#print(y)

(2100, 122)


In [ ]:
# Import label encoder 
from sklearn import preprocessing 
  
# label_encoder object knows how to understand word labels. 
label_encoder = preprocessing.LabelEncoder() 
  
# Encode labels in column 'species'. 
Y= label_encoder.fit_transform(Y) 

In [ ]:
Y

array([3, 3, 3, ..., 1, 5, 1])

In [ ]:
np.unique(Y)

array([0, 1, 2, 3, 4, 5, 6])

##Forward Propagration

In [ ]:
def forward_prop(params):
    """Forward propagation as objective function

    This computes for the forward propagation of the neural network, as
    well as the loss. It receives a set of parameters that must be
    rolled-back into the corresponding weights and biases.

    Inputs
    ------
    params: np.ndarray
        The dimensions should include an unrolled version of the
        weights and biases.

    Returns
    -------
    float
        The computed negative log-likelihood loss given the parameters
    """
    # Neural network architecture
    n_inputs = 122
    n_hidden = 30
    n_classes = 7

    #W1 = 122*30 
    #b1=30
    #w2=30*7
    #b2=7
    # Roll-back the weights and biases
    W1 = params[0:3660].reshape((n_inputs,n_hidden))
    b1 = params[3660:3690].reshape((n_hidden,))
    W2 = params[3690:3900].reshape((n_hidden,n_classes))
    b2 = params[3900:3907].reshape((n_classes,))

    # Perform forward propagation
    z1 = X.dot(W1) + b1  # Pre-activation in Layer 1
    a1 = np.tanh(z1)     # Activation in Layer 1
    z2 = a1.dot(W2) + b2 # Pre-activation in Layer 2
    logits = z2          # Logits for Layer 2

    # Compute for the softmax of the logits
    exp_scores = np.exp(logits)
    probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)

    # Compute for the negative log likelihood
    N = 2100 # Number of samples
    corect_logprobs = -np.log(probs[range(N), Y])
    loss = np.sum(corect_logprobs) / N

    return loss

In [ ]:
def f(x):
    """Higher-level method to do forward_prop in the
    whole swarm.

    Inputs
    ------
    x: numpy.ndarray of shape (n_particles, dimensions)
        The swarm that will perform the search

    Returns
    -------
    numpy.ndarray of shape (n_particles, )
        The computed loss for each particle
    """
    n_particles = x.shape[0]
    j = [forward_prop(x[i]) for i in range(n_particles)]
    return np.array(j)

##Training With PSO

In [ ]:
#@title Default title text
options = {'c1': 0.5, 'c2': 0.3, 'w':0.9}

# Call instance of PSO
dimensions = (122*30)+30+(30*7)+7
optimizer = ps.single.GlobalBestPSO(n_particles=50, dimensions=dimensions, options=options)

# Perform optimization
cost, pos = optimizer.optimize(f, iters=100)

2020-12-02 06:05:26,774 - pyswarms.single.global_best - INFO - Optimize for 100 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████|100/100, best_cost=0.229
2020-12-02 06:05:43,510 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.22928936894822288, best pos: [1.72205008 1.48970272 2.12859569 ... 2.64643201 2.72567058 2.6703691 ]


##Testing

In [ ]:
X_test=pd.concat([Normaldf.sample(2000), DoSdf.sample(100)], axis=0) #2100

In [ ]:
#X, X_test, y, y_test = sklearn.model_selection.train_test_split(X, y, test_size = 0.3, random_state = 2)

In [ ]:
#Dropping Class Labels
Y_test = X_test['label']
#Y = np.asarray(X['label'])#.to_numpy()
#CL_Train_R2L = Train_R2L['label']
#CL_Train_Probe = Train_Probe['label']
#CL_Train_U2R = Train_U2R['label']
#CL_Train_Mixed = Train_Mixed['label']

X_test = X_test.drop(['label'], axis=1)
#Train_R2L = Train_R2L.drop(['label'], axis=1)
#Train_Probe = Train_Probe.drop(['label'], axis=1)
#Train_U2R = Train_U2R.drop(['label'], axis=1)
#Train_Mixed = Train_Mixed.drop(['label'], axis=1)

In [ ]:
X_test = X_test.iloc[:, :].values
print(X_test.shape)

print(Y_test)

(2100, 122)
8511       normal
18347      normal
91109      normal
89602      normal
96200      normal
           ...   
114469    neptune
8338      neptune
15839     neptune
38698     neptune
11564       smurf
Name: label, Length: 2100, dtype: object


In [ ]:
# Import label encoder 
from sklearn import preprocessing 
  
# label_encoder object knows how to understand word labels. 
label_encoder = preprocessing.LabelEncoder() 
  
# Encode labels in column 'species'. 
Y_test= label_encoder.fit_transform(Y_test) 

In [ ]:
np.unique(Y_test)

array([0, 1, 2, 3, 4])

In [ ]:
def predict(X, params):
    """
    Use the trained weights to perform class predictions.

    Inputs
    ------
    X: numpy.ndarray
        Input Iris dataset
    pos: numpy.ndarray
        Position matrix found by the swarm. Will be rolled
        into weights and biases.
    """
    # Neural network architecture
    n_inputs = 122
    n_hidden = 30
    n_classes = 5


    # Roll-back the weights and biases
    W1 = params[0:3660].reshape((n_inputs,n_hidden))
    b1 = params[3660:3690].reshape((n_hidden,))
    W2 = params[3690:3840].reshape((n_hidden,n_classes))
    b2 = params[3840:3845].reshape((n_classes,))

    # Perform forward propagation
    z1 = X.dot(W1) + b1  # Pre-activation in Layer 1
    a1 = np.tanh(z1)     # Activation in Layer 1
    z2 = a1.dot(W2) + b2 # Pre-activation in Layer 2
    logits = z2          # Logits for Layer 2

    y_pred = np.argmax(logits, axis=1)
    return y_pred

In [ ]:
(predict(X_test, pos) == Y_test).mean()

0.0009523809523809524